In [19]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [20]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

print('문서 1과 문서 2의 유사도 :', cos_sim(doc1, doc2))
print('문서 1과 문서 3의 유사도 :', cos_sim(doc1, doc3))
print('문서 2과 문서 3의 유사도 :', cos_sim(doc2, doc3))

문서 1과 문서 2의 유사도 : 0.6666666666666667
문서 1과 문서 3의 유사도 : 0.6666666666666667
문서 2과 문서 3의 유사도 : 1.0000000000000002


In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory = False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [22]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

data.columns

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [24]:
data.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [26]:
#상위 2만개의 샘플을 data에 저장
data = data.head(20000)

In [27]:
#tf-idf를 연산할 때 데이터에 Null값이 들어있으면 에러가 발생.
#TF-IDF의 대상이 되는 data의 overview열에 결측값에 해당하는 Null값이 있는지 확인
#overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:', data['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [28]:
#결측값을 빈값으로 대체
data['overview'] = data['overview'].fillna('')

In [29]:
#overview열에 대해서 tf-idf행렬을 구한 후 행렬의 크기를 출력해본다.
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :', tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [30]:
#tf-idf행렬의 크기는 20,000의 행을 가지고 47,847의 열을 갖는 행렬.
# 20,000개의 영화를 표현하기 위해서 총 47,487개의단어가 사용되었음을 의미한다.
# 47,847차원의 문서 벡터가 20,000개가 존재한다고 표현할 수 있다.
# 20,000개의 문서 벡터에 대해서 상호간의 코사인 유사도를구한다.

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


코사인 유사도 연산 결과로는 20,000행 20,000열의 행렬을 얻는다.

이는 20,000개의 각 문서 벡터(영화 줄거리 벡터)와자기 자신을 포함한 20,000개의 문서 벡터 간의 유사도가 기록된 행렬이다.

모든 20,000개 영화의 상호 유사도가 기록되어져 있다.

이제 기존 데이터프레임으로부터 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리 title_to_index를 만들어둔다.

In [31]:
title_to_index = dict(zip(data['title'], data.index))

idx = title_to_index['Father of the Bride Part II']
print(idx)

4


선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만든다.

In [32]:
def get_recommendations(title, cosine_sim = cosine_sim):
    #선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    #해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    #유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse = True)

    #가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    #가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    #가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

In [33]:
get_recommendations('The Dark Knight Rises')

,title
12481,The Dark Knight
150,Batman Forever
1328,Batman Returns
15511,Batman: Under the Red Hood
585,Batman
9230,Batman Beyond: Return of the Joker
18035,Batman: Year One
19792,"Batman: The Dark Knight Returns, Part 1"
3095,Batman: Mask of the Phantasm
10122,Batman Begins
